In [3]:
import pandas as pd

# 读取原始文档，生成分段数据
- 原始文档为`fin_articles.csv`,包含列：src_web,keywords,text,len_text
- 按照换行符分割文档
- 分段数据包含：src_idx,keeywords,text,mentions

In [ ]:
# 准备语料
df=pd.read_csv(r'fin_articles.csv')
df.head(2)

In [ ]:
# 文本按换行符分割
all_src_idx=[]
all_kws=[]
all_texts=[]
for index,row in df.iterrows():
    texts=row['text'].split('\n') if isinstance(row['text'],str) else []
    # texts=[t for t in texts if len(t)>5] #过短的直接剔除
    all_src_idx.extend([index] * len(texts)) # 所属原文idx
    all_kws.extend([row['keywords']] * len(texts))
    all_texts.extend(texts)
print(len(all_texts))
# segment_articles
df_seg=pd.DataFrame({'src_idx':all_src_idx, 'keywords':all_kws, 'text':all_texts})
df_seg.head(20)


In [ ]:
# df_seg[df_seg['text'].str.len()<5].head()
df_seg=df_seg[df_seg['text'].str.len()>5] #过短的句子没有意义 空字符串会报错
df_seg.tail(10)

In [ ]:
# 删除一部分数据后，索引列变得不连续，重新索引
df_seg=df_seg.reset_index(drop=True)
df_seg.tail(10)

In [ ]:
df_seg.to_csv('segment_articles.csv',sep='\t')

# 读取存储的分段数据
- 注意指定index_col=0，文件里面包含了index
- 总数据量10w左右，后期需要进行一定筛选，此处先全部识别出来

In [7]:
df_seg2=pd.read_csv(r'..\data\segment_articles.csv',sep='\t',index_col=0)
df_seg2.head()

,src_idx,keywords,text,mentions
0,0,信息泄露,消费火正旺 金融添把柴,NaN
1,0,信息泄露,年轻人消费方式的改变和信用消费意识的增强，推动消费金融行业井喷式增长,NaN
2,0,信息泄露,“现在工作这么累，想在休息日好好犒劳自己。只要有假期，我就会去世界各地旅行，已经去过韩国、日...,NaN
3,0,信息泄露,80后、90后等年轻一代消费方式的改变和信用消费意识的增强，是近些年我国消费金融行业蓬勃发展...,NaN
4,0,信息泄露,银保监会近日下发通知，要求积极发展消费金融，适应多样化、多层次消费需求，提供和改进差异化金融...,NaN


In [10]:
# 切片赋值测试
# df_seg2.loc[1:3,'mentions']=['m1','m2','M3']# 切片为左右闭区间
# df_seg2.head()
def ner_partation(df,start,end, entity_list, col='mentions'):
    assert len(entity_list)==end-start+1
    df.loc[start:end,col]=entity_list
    return


In [35]:
import logging
logging.basicConfig(level=logging.INFO)
from tqdm import tqdm
# from stanzaNER import ner
import stanzaNER
import time

In [38]:
import importlib
importlib.reload(stanzaNER)

2022-01-22 21:54:28 INFO: "zh" is an alias for "zh-hans"
INFO:stanza:"zh" is an alias for "zh-hans"
2022-01-22 21:54:28 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package   |
-------------------------
| tokenize  | gsdsimp   |
| pos       | gsdsimp   |
| ner       | ontonotes |

INFO:stanza:Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package   |
-------------------------
| tokenize  | gsdsimp   |
| pos       | gsdsimp   |
| ner       | ontonotes |

2022-01-22 21:54:28 INFO: Use device: cpu
INFO:stanza:Use device: cpu
2022-01-22 21:54:28 INFO: Loading: tokenize
INFO:stanza:Loading: tokenize
2022-01-22 21:54:28 INFO: Loading: pos
INFO:stanza:Loading: pos
2022-01-22 21:54:29 INFO: Loading: ner
INFO:stanza:Loading: ner
2022-01-22 21:54:30 INFO: Done loading processors!
INFO:stanza:Done loading processors!


<module 'stanzaNER' from 'd:\\Projects\\finance_el\\preprocess\\stanzaNER.py'>

In [39]:
# max_line=df_seg2.shape[0]
max_line=2050 # 先测试一下2050行,耗时300.87秒
t_per_epoch=100
epochs = max_line // t_per_epoch + 1 # 101轮循环完
start_t=time.time()
for i in tqdm(range(epochs)):
    try:
        start = i*t_per_epoch
        end = min((i+1)*t_per_epoch,max_line)-1
        texts=df_seg2.loc[start:end, 'text'].tolist()
        entitys=stanzaNER.ner(texts)
        ner_partation(df_seg2,start,end, entitys, col='mentions')
    except Exception:
        logging.error('发生异常，当前start:{}, 耗时:{:.2f}秒'.format(start,time.time() - start_t))
print("执行成功，耗时: {:.2f}秒".format(time.time() - start_t))

100%|██████████| 21/21 [05:00<00:00, 14.33s/it]

执行成功，耗时: 300.87秒


In [40]:
df_seg2.head()

,src_idx,keywords,text,mentions
0,0,信息泄露,消费火正旺 金融添把柴,O
1,0,信息泄露,年轻人消费方式的改变和信用消费意识的增强，推动消费金融行业井喷式增长,O
2,0,信息泄露,“现在工作这么累，想在休息日好好犒劳自己。只要有假期，我就会去世界各地旅行，已经去过韩国、日...,厄瓜多尔/GPE;上海某律所/ORG;塞班岛/GPE;以色列/GPE;日本/GPE;缅甸/G...
3,0,信息泄露,80后、90后等年轻一代消费方式的改变和信用消费意识的增强，是近些年我国消费金融行业蓬勃发展...,沪江网校/ORG;天津/GPE;山东临沂/GPE;京东/GPE;刘明露/PERSON;淘宝/ORG
4,0,信息泄露,银保监会近日下发通知，要求积极发展消费金融，适应多样化、多层次消费需求，提供和改进差异化金融...,银保监会/ORG;中国人民大学重阳金融研究院/ORG;董希淼/PERSON


In [41]:
df_seg2.iloc[2047:2053]

,src_idx,keywords,text,mentions
2047,79,战略合作,紧随其后的是宝钢股份，在去年全年也大手笔委托银行理财产品，去年认购总金额已经达到490.2亿...,天茂集团/ORG;宝钢股份/ORG
2048,79,战略合作,另外，还有欧普照明等21家上市公司去年购买银行理财的金额均超过了100亿元。实际上，从这委托...,欧普照明/ORG
2049,79,战略合作,从认购记录中也能够发现，上市公司购买的理财产品主要分为三类：一是短期银行理财产品，一般期限在...,O
2050,79,战略合作,而随着银行理财收益率的不断下降，上市公司的闲置资金又去往何方？有业内人士分析称，随着A股市场...,NaN
2051,79,战略合作,(责编：勾雅文、李�P),NaN
2052,80,战略合作,平安健康（检测）中心进驻襄阳 高品质健康检测服务温暖冬季,NaN


In [44]:
from tqdm import tqdm
# from stanzaNER import ner_single
import stanzaNER
for index,row in tqdm(df_seg2.iterrows()):
    df_seg2.loc[index,'mentions']=stanzaNER.ner_single(row['text'])
df_seg2.head()

0it [00:00, ?it/s]


AttributeError: module 'stanzaNER' has no attribute 'ner_single'

In [ ]:
from stanzaNER import ner
import time
start_t=time.time()
all_texts=all_texts[:500]
all_entitys=ner(all_texts)
print("NER耗时: {:.2f}秒".format(time.time() - start_t))
df2=pd.DataFrame({'text':all_texts,'entitys':all_entitys})
df2.head()

In [ ]:
df2.to_csv('test_500.csv',index=True,sep='\t')